In [ ]:
import pandas as pd
import numpy as np
import json
from urllib2 import urlopen

import sys
sys.path.insert(0, '../src/')

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
df_path = "./../.data/FFChallenge_v2/background.dta"

with open(df_path, "r") as f:
    df_features = pd.read_stata(f)

df_train = df_features.set_index('challengeID')
print df_train.shape

# Convert to Numeric, If Possible
df_train_na = df_train.replace('NA', np.NaN)
df_train_na.cf4fint = pd.to_datetime(df_train_na.cf4fint)
df_train_na_cols = df_train_na.columns[df_train_na.dtypes == 'object']
df_train_na[df_train_na_cols] = df_train_na[df_train_na_cols].apply(lambda x: pd.to_numeric(x, errors = 'ignore'))
print df_train_na.shape

# Throw out what is still an object -- revisit later if needed.
df_train_no_obj = df_train_na[df_train_na.columns[df_train_na.dtypes != 'object']]
final_cols = df_train_no_obj.columns[~ df_train_no_obj.isnull().all()]
df_final = pd.DataFrame(df_train_no_obj[final_cols])
print df_final.shape

# Find number of unique values in each column. If unique == 1, then remove from final data frame.
n = df_final.apply(lambda x: len(x.unique()))
df_final = pd.DataFrame(df_final[df_final.columns[n>1]])
print df_final.shape

In [ ]:
# Finally, remove these because they're a pain in the ass.
garbage = [
    'm1f1c', 
    'cm1povca', 
    'cf1povca', 
    'm2b6', 
    'f4a10h2', 
    'f4a10h3',
    'p5i30a', 
    'p5h23', 
    'cm5saliva',
    'ck5saliva',
    'cf4fint',
]
mask = [x for x in df_final.columns if x not in garbage]
df_final = df_final[mask]
df_final.shape

In [ ]:
def clean_categories(x):
    if isinstance(x,basestring):
        x
        return int(x.split(' ')[0])
    else:
        return x

In [ ]:
# Next step... make sure categorical data is uniformly spaced...
df_clean = df_final.applymap(clean_categories)
df_clean.head()

### Create Brian Features

1. Categories that substitute f/m/pcg for equivalent questions (fill values in the following order).

        a. Fill in missing values using w/in survey questions.
        Solves the problem of different pathways to answering questions; e.g., sometimes same questions are asked depending on different circumstances they can be labeled differently.
        
        b. Fill in missing values using m-f questions (validated).
        Solves the problem of missing values for same year. Seems better than going multi-year away.
        
        c. Fill in missing values using cross-year questions (validated).
        Solves problem of survey drop-outs if no info available from mother on current condition.
        
        d. Freq. distribution across what is left column-wise.
    
    **Outcome: This will condense the parent categories into one representative category regardless of the PCG of the child.**
    
    
2. Averages / sums for particular questions.

        a. Eviction-Related Questions
        b. Job-training questions.
        c. materialHardship questions.

In [ ]:
import re

In [ ]:
def missingValuesReport(df):

    def getSurveyName(x):
        r = re.compile('([a-z]+[0-9])')
        m = r.match(x)
        if m:
            return m.groups()[0]
        else:
            return x

    nulls = (df < 0).sum()
    missing = nulls.sum()
    total = df.size
    print 'Num. Missing Values: {} of {}, {:.2%}\n'.format(missing, total, float(missing) / total)
    
    print 'Top Missing Surveys:'
    print nulls.groupby(by=getSurveyName).sum().sort_values(ascending=False).head(10)
    
missingValuesReport(df_clean)

### Step 1. W/in survey treatment.

In [ ]:
from ffcDataCleaning import InSurveyLookup
reload(InSurveyLookup)

In [ ]:
inSurveyLookup = InSurveyLookup.Lookup(master_list)

In [ ]:
df_step1 = df_clean.apply(lambda x: inSurveyLookup.fillColumn(df_clean,x.name))

In [ ]:
missingValuesReport(df_step1)

In [ ]:
df_step1.to_csv('../output/imputing/df_step1.csv')

### Step 2. Cross M-F Survey Treatment

In [ ]:
from ffcDataCleaning import MFLookup
reload(MFLookup)

In [ ]:
## dict with question on where child spends time:
withMF = {
    'm1': 'm1a11',
    'f1': 'm1a11',
    'm2': 'm2a3',
    'f2': 'f2a3',
    'm3': 'm3a2',
    'f3': 'f3a2',
    'm4': 'm4a2',
    'f4': 'f4a2',
    'm5': 'm5a2',
    'f5': 'f5a2',
}

In [ ]:
master_list = np.array(df_final.columns)
mfLookup = MFLookup.Lookup(master_list)
mfLookup.pairMatches.head()

In [ ]:
df_step2 = df_step1.apply(lambda x: mfLookup.fillColumn(df_step1,x.name))

In [ ]:
df_step2.to_csv('../output/imputing/.data/df_step2.csv')

In [ ]:
df_step2 = pd.read_csv('../output/imputing/.data/df_step2.csv',index_col=0)

In [ ]:
df_step2.head()
missingValuesReport(df_step2)

### Step 3. Cross Year Lookup

In [ ]:
from ffcDataCleaning import CrossYearLookup
reload(CrossYearLookup)

In [ ]:
cyLookup = CrossYearLookup.Lookup()

In [ ]:
cyLookup.pairMatches_father

In [ ]:
cyLookup.iterCount = 0
df_step3 = df_step2.apply(lambda x: cyLookup.fillColumn(df_step2,x.name))

In [ ]:
cyLookup.findSimilarCYSurvey('f3b3e_1')

In [ ]:
r = re.compile('(-+|.*)\s+([0-9]+\.[0-9]+|-)\s\s(.*)')
r = re.compile('(-+|.*\s+)([0-9]+\.[0-9]+\s\s|-\s\s|$)(.*)')
s0 = '-----------------------'
s1 = '  f1e1c2    -       People who currently live in your HH - 2nd gender?'
s2 = '* f2k9      29.0    Are you currently looking for a regular job?'
s3 = '  m1a16     -       Who does the baby looks like?'
m = r.match(s0)
print m.groups()
m = r.match(s1)
print m.groups()
m = r.match(s2)
print m.groups()
m = r.match(s3)
print m.groups()

In [ ]:
df_step3.to_csv('../output/imputing/.data/df_step3.csv')

In [ ]:
print df_step3.head(5)
missingValuesReport(df_step3)

In [ ]:
mfLookup.iterCount = 0
df_step4 = df_step3.apply(lambda x: mfLookup.fillColumn(df_step3,x.name))

In [ ]:
df_step4.to_csv('../output/imputing/.data/df_step4.csv')

In [ ]:
print missingValuesReport(df_step4)
df_step4

### Conclusions from Null Report:

The procedure removed just about all null values (except for only a few) in the m/f surveys. Will set the threshold for inclusion in the final data set to isnull < 50; and impute the rest (most frequent).

In [ ]:
nullReport = df_step4.isnull().sum().sort_values(ascending=False)
nullReport.to_csv('../output/nullReport.csv')

naReport = (df_step4 < 0).sum().sort_values(ascending=False)
naReport.to_csv('../output/naReport.csv')    

In [ ]:
# Remove Null Columns (keep nulls down to 50)..
null_threshold = 50
cols = df_step4.columns[(df_step4.isnull().sum() < neg_threshold)].tolist()
df_step5 = pd.DataFrame(df_step4[cols])

In [ ]:
# Remove Negative Columns (keep negatives down to 50)..
neg_threshold = 500
cols = df_step5.columns[((df_step5 < 0).sum() < neg_threshold)].tolist()
df_step6 = pd.DataFrame(df_step5[cols])

In [ ]:
print df_step5.shape
print df_step6.shape

### Finish imputing

In [ ]:
from sklearn.preprocessing import imputation

In [ ]:
im = imputation.Imputer(strategy="most_frequent")
col = df_step6.columns
idx = df_step6.index
imputed = pd.DataFrame(im.fit_transform(df_step6.values), index = idx, columns = col)

In [ ]:
imputed.to_csv('../output/imputing/.data/imputed.csv')

### Custom Features

#### Material Hardship and Eviction (11 Questions):

In [ ]:
free_food = {
    '5': ['m5f23a', 'f5f23a', 'n5g1a'],
    '4': ['m4i23a', 'f4i23a'],
    '3': ['m3i23a', 'f3i23a'],
    '2': ['m2h19a', 'f2h17a'],
}

no_food = {
    '5': ['m5f23b', 'f5f23b', 'n5g1b'],
    '4': ['m4i23c', 'f4i23c'],
    '3': ['m3i23a', 'f3i23a'],
    '2': ['m2h19c', 'f2h17c'],
}

full_rent = {
    '5': ['m5f23c',
    'f5f23c',
    'n5g1c'],
    
    '4': ['m4i23d',
    'f4i23d'],

    '3': ['m3i23b',
    'f3i23b'],

    '2': ['m2h19d',
    'f2h17d'],
}

evicted = {
    '5': ['m5f23d',
    'f5f23d',
    'n5g1d'],
    
    '4': ['m4i23e',
    'f4i23e'],

    '3': ['m3i23c',
    'f3i23c'],
    
    '2': ['m2h19e',
    'f2h17e'],
}

gas_bill = {
    '5': ['m5f23e',
    'f5f23e',
    'n5g1e'],
    
    '4': ['m4i23f',
    'f4i23f'],
    
    '3': ['m3i23d',
    'f3i23d'],
    
    '2': ['m2h19f',
    'f2h17f'],
}

turn_off_oil = {
    '5': ['m5f23f',
    'f5f23f',
    'n5g1f'],
    '4': ['m4i23g',
    'f4i23g'],
    '3': ['m3i6f',
    'f3i6f'],
    '2': ['m2h19g',
    'f2h17g'],
}

borrow_money = {
    '5': ['m5f23g',
    'f5f23g',
    'n5g1g'],
    
    '4': ['m4i23h',
    'f4i23h'],
    
    '3':['m3i23e',
    'f3i23e'],
    
    '4': ['m2h19i',
    'f2h17i'],
}

move_in_financial = {
    '5': ['m5f23h',
    'f5f23h',
    'n5g1h'],
    
    '4': ['m4i23i',
    'f4i23i'],

    '3': ['m3i23f',
    'f3i23f'],
    
    '2': ['m2h19j',
    'f2h17j'],
}

shelter = {
    '5': ['m5f23i',
    'f5f23i',
    'n5g1i'],
    
    '4': ['m4i23j',
    'f4i23j'],
    
    '3': ['m3i23g',
    'f3i23g'],
    
    '2': ['m2h19k',
    'f2h17k'],
}

no_doctor = {
    '5': ['m5f23j',
    'f5f23j',
    'n5g1j'],
    
    '4': ['m4i23k',
    'f4i23k'],
    
    '3': ['m3i23h',
    'f3i23h'],
    
    '2': ['m2h19l',
    'f2h17l'],
}

phone_canceled = {
    '5': ['m5f23k',
    'f5f23k'],

    '4': ['m4i23n',
    'f4i23n'],
    
    '3': ['m3i6a',
    'f3i6a'],
    
    '2': ['m2h19h',
    'f2h17h'],
}

In [ ]:
mtlHardship = {
    'free_food': free_food,
    'no_food': no_food,
    'full_rent': full_rent,
    'evicted': evicted,
    'gas_bill': gas_bill,
    'turn_off_oil': turn_off_oil,
    'borrow_money': borrow_money,
    'move_in_financial': move_in_financial,
    'shelter': shelter,
    'no_doctor': no_doctor,
    'phone_canceled': phone_canceled,
}

mtlHardship_list = [j for v in mtlHardship.itervalues() for i in v.itervalues() for j in i]
marriage_list = {'{}_{}'.format(k1,k2):j for k1, v in mtlHardship.iteritems() for k2, j in v.iteritems()}
print marriage_list

In [ ]:
df_mtlHardship = pd.DataFrame(df_clean[mtlHardship_list])
df_mtlHardship[df_mtlHardship == 2] = 0.
df_mtlHardship[df_mtlHardship > 0] = 1.

df_mtlHardship_agg = pd.DataFrame()
for k,v in marriage_list.iteritems():
    df_mtlHardship_agg[k] = df_mtlHardship[v].apply(lambda x: x[x >=0].mean() ,axis=1)

df_mtlHardship_agg = df_mtlHardship_agg.apply(fixNan, axis=1)
df_mtlHardship_agg[df_mtlHardship_agg.isnull()] = 0

In [ ]:
col_n = df_mtlHardship_agg.columns.map(lambda x: int(x.split('_')[-1]))
def filt_year_n(x,n):
    n = int(n)
    return x.index[col_n == n].tolist()

df_mtlHardship_agg_total = pd.DataFrame()
for n in range(2,6):
    print n
    df_mtlHardship_agg_total['total_{}'.format(n)] = df_mtlHardship_agg.apply(lambda x: x[filt_year_n(x,n)].sum(), axis=1)

In [ ]:
df_mtlHardship_agg_total

In [ ]:
df_mtlHardship_agg.to_csv('../output/imputing/.data/materialHardship_features.csv')
df_mtlHardship_agg_total.to_csv('../output/imputing/.data/materialHardshipTotal_features.csv')

In [ ]:
print df_mtlHardship.shape
print df_mtlHardship_agg.shape

In [ ]:
for K,V in mtlHardship:
    {(k, df_clean[v].apply()) for k,v in V}
        

#### Grit (average of 4 questions)

1. I keep at my schoolwork until I'm done with it.
2. I make a plan to get something done, I stick with it.
3. I finish whatever I begin.
4. I am a hard worker.

Note: Not much to do about missing values here... just see if you can apply as is and guess on the rest.

In [ ]:
grit = [
    'k5g1a',
    'k5g1b',
    'k5g1c',
    'k5g1d',
    'k5g1e',
]

In [ ]:
print 'Num missing: {}'.format(df_clean[grit].apply(lambda x: (x > 0).all(), axis = 1).sum())
print 'Num total: {}'.format(len(df_clean))

In [ ]:
df_grit = df_clean[grit]
df_grit = df_grit.apply(fixNan,axis=1)
mean = df_grit[(df_grit>=0).all(axis=1)].mean()

df_grit_final = pd.DataFrame(df_grit,copy=True)
for col in grit:
    df_grit_final[col].loc[df_grit_final[col]<0] = mean[col]

In [ ]:
df_grit_final.to_csv('../output/imputing/.data/df_grit.csv')

#### Layoff

1. Since {last time}, have you ever been laid off?

In [ ]:
from itertools import chain

In [ ]:
unemployment = {
    '5': ['m5f8a3',
    'f5f8a3'],
    '4': ['m4i8a3',
    'f4i8a3'],
    '3': ['m3i8a3',
    'f3i8a3'],
    '2': ['m2h9a3',
    'f2h8a3'],
}
unemployment_list = list(chain(*[v for k,v in unemployment.items()]))
print unemployment_list

In [ ]:
df_final[unemployment_list]

In [ ]:
def fixNan(x):
    x[x.isnull()] = x[~x.isnull()].mean()
    return x

df_layoff = pd.DataFrame(df_clean[unemployment_list])
df_layoff[df_layoff == 2] = 0.
df_layoff[df_layoff > 0] = 1.

df_layoff_agg = pd.DataFrame()
for k,v in unemployment.iteritems():
    df_layoff_agg['layoff_{}'.format(k)] = df_layoff[v].apply(lambda x: x[x >=0].mean() ,axis=1)

df_layoff_agg = df_layoff_agg.apply(fixNan, axis=1)
df_layoff_agg[df_layoff_agg.isnull()] = 0

In [ ]:
df_layoff_agg.isnull().sum()
df_layoff_agg.to_csv('../output/imputing/.data/layoff_features.csv')

#### Job Training

1. Ever taken job skills/training classes?

In [ ]:
job_skills = {
    '5': ['m5i1',
    'm5i3b',
    'f5i1',
    'f5i3b'],
    
    '4': ['m4k1',
    'm4k3b',
    'f4k1',
    'f4k3b'],
    
    '3': ['m3k1',
    'm3k3b',
    'f3k1',
    'f3k3b'],
    
    '2': ['m2k1',
    'm2k3a13',
    'f2k2',
    'f2k5a13'],
}

jobSkill_list = list(chain(*[v for k,v in job_skills.items()]))
print jobSkill_list

In [ ]:
df_final[jobSkill_list]

In [ ]:
def fixNan(x):
    x[x.isnull()] = x[~x.isnull()].mean()
    return x

df_jobSkill = pd.DataFrame(df_clean[unemployment_list])
df_jobSkill[df_jobSkill == 2] = 0.
df_jobSkill[df_jobSkill > 0] = 1.

df_jobSkill_agg = pd.DataFrame()
for k,v in unemployment.iteritems():
    df_jobSkill_agg['jobSkill_{}'.format(k)] = df_jobSkill[v].apply(lambda x: x[x >=0].mean() ,axis=1)

df_jobSkill_agg = df_jobSkill_agg.apply(fixNan, axis=1)
df_jobSkill_agg[df_jobSkill_agg.isnull()] = 0

In [ ]:
df_jobSkill_agg.isnull().sum()
df_jobSkill_agg.to_csv('../output/imputing/.data/jobSkill_features.csv')

#### GPA

1. no precedent
2. Probably look at whatever scores are available...

# Second Imputed Set... Naive.

In [ ]:
df_final

In [ ]:
from sklearn.preprocessing import LabelEncoder, imputation

In [ ]:
imputed = pd.read_csv('../output/imputing/.data/df_step4.csv', index_col=0)

In [ ]:
imputed[imputed < 0] = np.nan

In [ ]:
imputed = imputed[imputed.columns[~imputed.isnull().all()].tolist()]

In [ ]:
im = imputation.Imputer(strategy="most_frequent")
col = imputed.columns
idx = imputed.index
imputed2 = pd.DataFrame(im.fit_transform(imputed.values), index = idx, columns = col)

In [ ]:
imputed2.shape

In [ ]:
imputed2.to_csv('../output/imputing/.data/imputed_nodrops.csv')

In [ ]:
labelMaker = LabelEncoder()

In [ ]:
mask = df_final.columns[df_final.dtypes.apply(lambda x: x.name == 'category')].tolist()
mask = list(set(mask) & set(imputed2.columns))

In [ ]:
imputed3 = pd.DataFrame(imputed2,copy=True)
imputed3[mask] = imputed3[mask].apply(labelMaker.fit_transform)

In [ ]:
imputed3.shape

In [ ]:
imputed3.to_csv('../output/imputing/.data/imputed_nodrops-v2.csv')